In [1]:
#  Import dependencies
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.offline as py

# read data set into pandas dataframes
df_vax = pd.read_csv('/Users/jamesswank/Downloads/CIIS_Deidentified.csv')
# print unique value counts for each column
df_vax.nunique()

ID                       5166
vaccination_date          576
dosage_num                  4
vaccination_code            5
age_at_1stvaccination      83
race_ethnicity             12
patient_county              6
dtype: int64

In [2]:
# print unique values for counties
print(np.unique(df_vax['patient_county'].values))

['ADAMS' 'ARAPAHOE' 'DOUGLAS' 'adams' 'arapahoe' 'douglas']


In [3]:
# print unique values for vaccination codes
print(np.unique(df_vax['vaccination_code'].values))

['COVID-19 UF' 'COVID-19 Vector-NR (ASZ)' 'COVID-19 Vector-NR (JSN)'
 'COVID-19 mRNA (MOD)' 'COVID-19 mRNA (PFR)']


In [4]:
# change vacciantion date from string to datetime and print column data types to verify
df_vax['vaccination_date'] = pd.to_datetime(df_vax['vaccination_date'])
df_vax.dtypes


ID                                int64
vaccination_date         datetime64[ns]
dosage_num                        int64
vaccination_code                 object
age_at_1stvaccination             int64
race_ethnicity                   object
patient_county                   object
dtype: object

In [5]:
# standardize counties with all caps
df_vax['patient_county'] = df_vax['patient_county'].str.upper()
print(np.unique(df_vax['patient_county'].values))

['ADAMS' 'ARAPAHOE' 'DOUGLAS']


In [6]:
# verify unique date and county counts
df_vax.nunique()

ID                       5166
vaccination_date          294
dosage_num                  4
vaccination_code            5
age_at_1stvaccination      83
race_ethnicity             12
patient_county              3
dtype: int64

In [23]:
# Table 1. Calculate total individuals vaccinated by county (Adams, Arapahoe, Douglas)

# County population estimates for 2019
adams_pop = 517885
arap_pop = 656822
doug_pop = 351528



# Create dataframes for each county, and selecting for dose = 1 or 2
df_adams = df_vax.loc[(df_vax['patient_county'] == 'ADAMS') & (df_vax['dosage_num'] < 3)]
df_douglas = df_vax.loc[(df_vax['patient_county'] == 'DOUGLAS') & (df_vax['dosage_num'] < 3)]
df_arapahoe = df_vax.loc[(df_vax['patient_county'] == 'ARAPAHOE') & (df_vax['dosage_num'] < 3)]

# Select number of unique IDs for each county
data = {'COUNTY':['ADAMS', 'DOUGLAS', 'ARAPAHOE'],
       'Total Vaccinated Individuals':[
           df_adams['ID'].nunique(),
           df_douglas['ID'].nunique(),
           df_arapahoe['ID'].nunique(),
       ]}
        
total_vaccinated_by_county = pd.DataFrame(data)
total_vaccinated_by_county

# import population by age group data to dataframe
# pop_by_age_group = pd.read_csv('/Users/jamesswank/Downloads/county_sya_data.csv', thousands=',')


,COUNTY,Total Vaccinated Individuals
0,ADAMS,1828
1,DOUGLAS,1532
2,ARAPAHOE,1806


In [55]:
# Calculate percent complete vaccination by county

# Vaccine list
vax_lst=['COVID-19 mRNA (MOD)', 'COVID-19 mRNA (PFR)']
    
# Select count of IDs by county where dosage = 2 and vaccine is MOD or PFR or JJ, divided by the 2019 pop.
data = {'COUNTY':['ADAMS', 'DOUGLAS', 'ARAPAHOE'],
       'Percent Complete Vaccinations by County':[
           (df_adams.ID[(df_adams.dosage_num == 2) & (df_adams.vaccination_code.isin(vax_lst)) | (df_adams.vaccination_code == 'COVID-19 Vector-NR (JSN)')].count()) / adams_pop,
           (df_douglas.ID[(df_douglas.dosage_num == 2) & (df_douglas.vaccination_code.isin(vax_lst)) | (df_douglas.vaccination_code == 'COVID-19 Vector-NR (JSN)')].count()) / doug_pop,
           (df_arapahoe.ID[(df_arapahoe.dosage_num == 2) & (df_arapahoe.vaccination_code.isin(vax_lst)) | (df_arapahoe.vaccination_code == 'COVID-19 Vector-NR (JSN)')].count()) / arap_pop   
       ]}

Pct_compl_by_cty = pd.DataFrame(data)
Pct_compl_by_cty

,COUNTY,Percent Complete Vaccinations by County
0,ADAMS,0.003277
1,DOUGLAS,0.004148
2,ARAPAHOE,0.002584


In [28]:
# Calculate percent of individuals vaccinated by age group (12-17, 18-64, 65+)

vax_lst_two=['COVID-19 mRNA (MOD)', 'COVID-19 mRNA (PFR)']
pop_by_age_group = pd.read_csv('/Users/jamesswank/Downloads/county_sya_data.csv', thousands=',')

data = {'Age Group':['12-17', '18-64', '65+'],
       '% Initiated':[
           ((df_vax.ID[(df_vax.dosage_num == 1) & (df_vax.age_at_1stvaccination > 12) & (df_vax.age_at_1stvaccination < 18)].count()) / pop_by_age_group.TOTAL[0]),
           ((df_vax.ID[(df_vax.dosage_num == 1) & (df_vax.age_at_1stvaccination > 17) & (df_vax.age_at_1stvaccination < 65)].count()) / pop_by_age_group.TOTAL[1]),
           ((df_vax.ID[(df_vax.dosage_num == 1) & (df_vax.age_at_1stvaccination > 64)].count()) / pop_by_age_group.TOTAL[2]),
       ],
       '% Completed':[
           ((df_vax.ID[(df_vax.dosage_num == 2) & ((df_vax.vaccination_code.isin(vax_lst_two)) | (df_vax.vaccination_code == 'COVID-19 Vector-NR (JSN)')) & (df_vax.age_at_1stvaccination > 12) & (df_vax.age_at_1stvaccination < 18)].count()) / pop_by_age_group.TOTAL[0]),
           ((df_vax.ID[(df_vax.dosage_num == 2) & ((df_vax.vaccination_code.isin(vax_lst_two)) | (df_vax.vaccination_code == 'COVID-19 Vector-NR (JSN)')) & (df_vax.age_at_1stvaccination > 17) & (df_vax.age_at_1stvaccination < 65)].count()) / pop_by_age_group.TOTAL[1]),
           ((df_vax.ID[(df_vax.dosage_num == 2) & ((df_vax.vaccination_code.isin(vax_lst_two)) | (df_vax.vaccination_code == 'COVID-19 Vector-NR (JSN)')) & (df_vax.age_at_1stvaccination > 64)].count()) / pop_by_age_group.TOTAL[2]),
       ]}

Pct_compl_by_age = pd.DataFrame(data)
Pct_compl_by_age.style.set_table_attributes("style='display:inline'").set_caption('% Vaccinated By Age Group')

,Age Group,% Initiated,% Completed
0,12-17,0.0113009,0.0103304
1,18-64,0.0120871,0.0105523
2,65+,0.0110061,0.0102545


In [59]:
# Calculate total booster vaccines by county, by vaccine code

df_booster = df_vax
df_booster = df_booster.drop(['ID', 'vaccination_date', 'age_at_1stvaccination', 'race_ethnicity'], axis=1)

# Create dataframe by selecting rows by MOD and PFR
df_booster_I = df_booster.loc[(df_booster['vaccination_code'] == 'COVID-19 mRNA (MOD)') | (df_booster['vaccination_code'] == 'COVID-19 mRNA (PFR)')]

#  Select for dosage = 3
df_booster_I = df_booster.loc[(df_booster['dosage_num'] == 3)]

#  Rename columns 
df_booster_final = df_booster_I.rename({'patient_county': 'COUNTY', 'vaccination_code':'VACCINE', 'dosage_num':'TOTAL'}, axis=1)

# Group by County, Vaccine and get counts
df_booster_final=df_booster_final.groupby(['COUNTY','VACCINE']).count()

pd.DataFrame(df_booster_final)
df_booster_final.style.set_caption('Boosters By County and Vaccine Code')


In [60]:
# Create dataframe for graph

# import graphing dependencies
import plotly.io as pio
import plotly.express as px
import plotly.offline as py

# create dataframe of completed vaccinations, 2 doses for Moderna and Phizer, 1 for JJ

df_vax_by_date = df_vax[(df_vax.dosage_num == 2) & ((df_vax.vaccination_code.isin(vax_lst)) | (df_vax.vaccination_code == 'COVID-19 Vector-NR (JSN)' ))]
df_vax_by_date = df_vax_by_date.sort_values('vaccination_date')
df_vax_by_date

,ID,vaccination_date,dosage_num,vaccination_code,age_at_1stvaccination,race_ethnicity,patient_county
3788,320365,2021-01-04,2,COVID-19 mRNA (PFR),38,White - Non Hispanic,ARAPAHOE
7599,206,2021-01-05,2,COVID-19 mRNA (PFR),31,White - Non Hispanic,ADAMS
2981,319943,2021-01-06,2,COVID-19 mRNA (PFR),35,White - Non Hispanic,ARAPAHOE
9937,1796,2021-01-06,2,COVID-19 mRNA (PFR),57,White - Non Hispanic,ADAMS
9250,1296,2021-01-06,2,COVID-19 mRNA (PFR),42,White - Non Hispanic,ADAMS
5658,741079,2021-01-06,2,COVID-19 mRNA (PFR),33,White - Non His,DOUGLAS
5239,740863,2021-01-07,2,COVID-19 mRNA (PFR),33,White - Non His,DOUGLAS
2619,319485,2021-01-07,2,COVID-19 mRNA (PFR),31,White - Non Hispanic,ARAPAHOE
3691,320316,2021-01-07,2,COVID-19 mRNA (PFR),40,White - Non Hispanic,ARAPAHOE
8097,692,2021-01-07,2,COVID-19 mRNA (PFR),33,White - Non Hispanic,ADAMS


In [53]:
# Create graph: Percent fully vaccinated over time

tot_pop = 1526235

# Group data by date and dosage, count by ID
df = df_vax_by_date.groupby(['vaccination_date', 'dosage_num'])['ID'].count().reset_index(name='count')

# Create cumulative sum column
df['cumsum'] = df['count'].cumsum()

# Create Vaccination percentage column
df['vax_pct'] = df['cumsum'].apply(lambda x: x / tot_pop)

# Create graph
fig = px.scatter(df, x=df.vaccination_date, y=df.vax_pct,
                 title='Percent Fully Vaccinated Over Time',
                 labels={
                    'vaccination_date': 'Vaccination Date',
                    'vax_pct': 'Percent Vaccinated'
                })
fig

In [48]:
# print dataframe that is being graphed to make sure the cumulative sum column looks correct

df = df_vax_by_date.groupby(['vaccination_date', 'dosage_num'])['ID'].count().reset_index(name='count')
df['cumsum'] = df['count'].cumsum()
df['vax_pct'] = df['cumsum'].apply(lambda x: x / tot_pop)
df


,vaccination_date,dosage_num,count,cumsum,vax_pct
0,2021-01-02,2,1,1,6.552071e-07
1,2021-01-04,2,1,2,1.310414e-06
2,2021-01-05,2,1,3,1.965621e-06
3,2021-01-06,2,4,7,4.586450e-06
4,2021-01-07,2,19,26,1.703538e-05
5,2021-01-08,2,19,45,2.948432e-05
6,2021-01-09,2,11,56,3.669160e-05
7,2021-01-10,2,2,58,3.800201e-05
8,2021-01-11,2,7,65,4.258846e-05
9,2021-01-12,2,2,67,4.389888e-05
